

Two most common types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**. 



## Collaborative Filtering

In general, Collaborative filtering (CF) is more commonly used than content-based systems because it usually gives better results and is relatively easy to understand (from an overall implementation perspective). The algorithm has the ability to do feature learning on its own, which means that it can start to learn for itself what features to use. 

CF can be divided into **Memory-Based Collaborative Filtering** and **Model-Based Collaborative filtering**. 

Here, we will implement Model-Based CF by using singular value decomposition (SVD) and Memory-Based CF by computing cosine similarity. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("infi_order_item.csv")

In [ ]:
df.head()

In [3]:
df[["customer_number"]].nunique()

customer_number    1297
dtype: int64

In [4]:
df[["product_id"]].nunique()

product_id    1456
dtype: int64

In [5]:
df[["order_id"]].nunique()

order_id    2157
dtype: int64

Performing Some Data Cleaning and Preprocessing

In [6]:
#preprocessing the phone numbers from customer without 977 or +
def phoneNumberStandardizer(num):
    num = str(num)
    num = num.replace("-","")
    if len(num) >10:
        num = num[-10:]
    return num

In [7]:
df["customer_number"]  = df["customer_number"].apply(phoneNumberStandardizer)

In [ ]:
df[["customer_number"]]

In [8]:
newdf = df[["customer_number","product_id"]]

In [9]:
final = newdf.groupby(["customer_number","product_id"]).size()

In [ ]:
final.to_frame()

In [10]:
final.to_frame().to_csv("finaldf.csv")

In [11]:
dfd = pd.read_csv("finaldf.csv")

In [ ]:
dfd.head()

In [12]:
df2 = dfd.rename(columns = {'0': 'Frequency'}, inplace = False)

In [ ]:
df2.head()

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
mms = MinMaxScaler((1,5))
#change range to 0 to 5 when trained on all products

In [15]:
df2[['Frequency']] = mms.fit_transform(df2[['Frequency']])

In [ ]:
df2.head()

In [16]:
df2['Frequency'].max()

5.0

In [ ]:
"""
Now Frequency can be rated as Rating
"""


In [17]:
final_data = df2.rename(columns = {'Frequency': 'ratings'}, inplace = False)

In [ ]:
final_data.head()

Now Making the matrix for matrix Factorization.

In [18]:
to_train=final_data.pivot_table(index="customer_number",columns="product_id",values="ratings")

In [ ]:
to_train.head()

In [19]:
fill=to_train.fillna(0)

In [ ]:
fill

In [20]:
n_customers = final_data.customer_number.nunique()
n_products = final_data.product_id.nunique()

In [21]:
print(f"customers:{n_customers} products:{n_products}")

customers:1284 products:1456


In [22]:
train_arr =fill.values


## Train Test Split

Recommendation Systems by their very nature are very difficult to evaluate, but to evaluate them here---> we'll split our data into two sets. However, we won't do our classic X_train,X_test,y_train,y_test split. Instead we can actually just segement the data into two sets of data:

In [23]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(train_arr,test_size=0.25)

## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. 

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

* *Item-Item Collaborative Filtering*: “Users who liked this item also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a customer-product matrix which built from the entire dataset.

Since we have split the data into testing and training we will need to create two  matrices (all customers by all products). 

The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.  

After you have built the customer-product matrix you calculate the similarity and create a similarity matrix. 

The similarity values between items in *product-product Collaborative Filtering* are measured by observing all the users who have rated both items.  

For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.

A distance metric commonly used in recommender systems is *cosine similarity*, where the ratings are seen as vectors in ``n``-dimensional space and the similarity is calculated based on the angle between these vectors. 
Cosine similiarity for users *a* and *m* can be calculated using the formula below, where you take dot product of  the user vector *$u_k$* and the user vector *$u_a$* and divide it by multiplication of the Euclidean lengths of the vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

To calculate similarity between items *m* and *b* you use the formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Your first step will be to create the user-item matrix. Since you have both testing and training data you need to create two matrices.  

In [24]:
train_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
test_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

You can use the [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [26]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data, metric='cosine')
item_similarity = pairwise_distances(train_data.T, metric='cosine')

Next step is to make predictions. You have already created similarity matrices: `user_similarity` and `item_similarity` and therefore you can make a prediction by applying following formula for user-based CF:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

You can look at the similarity between users *k* and *a* as weights that are multiplied by the ratings of a similar user *a* (corrected for the average rating of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict. 

The idea here is that some users may tend always to give high or low ratings to all movies. The relative difference in the ratings that these users give is more important than the absolute values. To give an example: suppose, user *k* gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user *t* rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently. 

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [27]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [28]:
item_prediction = predict(train_data, item_similarity, type='item')
user_prediction = predict(train_data, user_similarity, type='user')

In [29]:
item_prediction

array([[0.00072723, 0.00069321, 0.00070027, ..., 0.00068729, 0.00068776,
        0.00068776],
       [0.0169627 , 0.01900853, 0.01852486, ..., 0.01947308, 0.01948647,
        0.01948647],
       [0.00149482, 0.00157128, 0.00158728, ..., 0.00155785, 0.00155892,
        0.00155892],
       ...,
       [0.00072723, 0.00069321, 0.00070027, ..., 0.00068729, 0.00068776,
        0.00068776],
       [0.01581382, 0.01525066, 0.01510738, ..., 0.01512027, 0.01513067,
        0.01513067],
       [0.02753073, 0.02894531, 0.02918371, ..., 0.02982818, 0.02984869,
        0.02984869]])

### Evaluation
There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

You can use the [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) function from `sklearn`, where the RMSE is just the square root of MSE. To read more about different evaluation metrics you can take a look at [this article](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with `prediction[ground_truth.nonzero()]`. 

In [30]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [31]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data)))

User-based CF RMSE: 1.0342570302498548
Item-based CF RMSE: 1.0463916783717733


Memory-based algorithms are easy to implement and produce reasonable prediction quality. 
The drawback of memory-based CF is that it doesn't scale to real-world scenarios and doesn't address the well-known cold-start problem, that is when new user or new item enters the system. Model-based CF methods are scalable and can deal with higher sparsity level than memory-based models, but also suffer when new users or items that don't have any ratings enter the system.

# Model-based Collaborative Filtering

Model-based Collaborative Filtering is based on **matrix factorization (MF)** which has received greater exposure, mainly as an unsupervised learning method for latent variable decomposition and dimensionality reduction. Matrix factorization is widely used for recommender systems where it can deal better with scalability and sparsity than Memory-based CF. The goal of MF is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items. 
When you have a very sparse matrix, with a lot of dimensions, by doing matrix factorization you can restructure the  user-item matrix into low-rank structure, and you can represent the matrix by the multiplication of two low-rank matrices, where the rows contain the latent vector. You fit this matrix to approximate your original matrix, as closely as possible, by multiplying the low-rank matrices together, which fills in the entries missing in the original matrix.

In [32]:
sparsity=round(1.0-len(df)/float(n_customers*n_products),3)
print('The sparsity level of dataset is ' +  str(sparsity*100) + '%')

The sparsity level of dataset is 99.6%


### SVD
A well-known matrix factorization method is **Singular value decomposition (SVD)**. Collaborative Filtering can be formulated by approximating a matrix `X` by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations, for more information I recommend to read articles: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 


Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.


Now you can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.



In [33]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print('User-based CF MSE: ' + str(rmse(X_pred, test_data)))

User-based CF MSE: 1.0412217017902896


In [34]:
X_pred

array([[-1.35611153e-03, -2.48318816e-04,  5.86703090e-04, ...,
        -4.76279292e-20,  5.27845919e-20,  5.27845919e-20],
       [ 1.04429879e-01, -1.23768576e-03,  1.27646461e-01, ...,
         1.55056274e-17,  3.83323384e-19,  3.83323384e-19],
       [-5.82807874e-03, -1.50129853e-03, -1.48757795e-02, ...,
        -2.48184778e-18, -1.37641707e-19, -1.37641707e-19],
       ...,
       [-3.82292559e-06,  6.96396132e-06, -3.49268824e-05, ...,
        -5.05461940e-21, -2.00829455e-21, -2.00829455e-21],
       [ 6.51056179e-02, -2.60324384e-02,  2.89696588e-02, ...,
        -1.56617719e-17,  3.95913057e-18,  3.95913057e-18],
       [-1.47742198e-01,  7.70923948e-02, -2.76238054e-03, ...,
        -1.93892361e-17,  2.03158993e-18,  2.03158993e-18]])

# REFERENCES

1. https://realpython.com/build-recommendation-engine-collaborative-filtering/
2.https://en.wikipedia.org/wiki/Collaborative_filtering#Hybrid
(Wikipedia for only mathematical concepts and animations)
3. https://towardsdatascience.com/paper-summary-matrix-factorization-techniques-for-recommender-systems-82d1a7ace74

4. https://www.youtube.com/watch?v=ZspR5PZemcs&t=808s
5. https://www.youtube.com/watch?v=mBcLRGuAFUk